In [1]:

from keras import backend as K
from keras.models import load_model
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import cv2
import time

import pickle

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
model_path = "jaad-bb-lstm-model.h5"
model = load_model(model_path)

In [3]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

In [4]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')

In [5]:
from jaad_data import JAAD

In [6]:
imdb = JAAD(data_path = jaad_path)

In [86]:
vid = "video_0002"

In [87]:
anno = imdb._get_annotations(vid)

In [88]:
bb_cross={}
ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]
print(ped_b)
for i, p in enumerate(ped_b):
    if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
        bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
#print(bb_cross)

['0_2_5b', '0_2_6b']


In [89]:
lstm_seq =  bb_cross[ped_b[0]][0:3]

In [90]:
df = pd.DataFrame(lstm_seq)
df = df.astype('float32')

In [91]:
df.loc[0:0, :] = np.array([0] * 4) 

In [92]:
df

,0,1,2,3
0,0.0,0.0,0.0,0.0
1,1515.0,638.0,1605.0,932.0
2,1515.0,638.0,1605.0,932.0


In [93]:
pickle_in = open("min-max-scaler.pkl","rb")
scaler = pickle.load(pickle_in)
print(type(scaler))
print(scaler.data_max_)
print(scaler.data_min_)

<class 'sklearn.preprocessing.data.MinMaxScaler'>
[1900.  953. 1919. 1079.]
[0. 0. 0. 0.]


In [94]:
global batch

In [95]:
batch = scaler.transform(df)

In [96]:
batch

array([[0.  , 0.  , 0.  , 0.  ],
       [0.8 , 0.67, 0.84, 0.86],
       [0.8 , 0.67, 0.84, 0.86]])

In [97]:
def play():
    global batch
    img_width, img_height = 960, 540
    start_time_video = time.time()
    clip_path = "/home/kara9147/JAAD/JAAD_clips/" + vid + ".mp4"
    cap = cv2.VideoCapture(clip_path)
    
    # Time to read all frames, predict and put bounding boxes around them, and show them.
    i = 0
    total_time = 0.0

    # Capture frame-by-frame
    ret = True
    while(ret):
        ret, origimg = cap.read()
        i = i + 1
        #print("Processing {} th frame".format(i))
        if (ret != False):
            #print(img.shape)
            img = cv2.resize(origimg, (img_width, img_height))
            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            print(y_hat)

            batch =  np.delete(batch, 0, axis = 0)
            batch =  np.append(batch, y_hat, axis = 0)
            print(batch)
            
            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # 4: Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            print(inv_yhat)
            
            
            
            np.set_printoptions(precision=2, suppress=True, linewidth=90)
            # print("Predicted boxes:\n")
            # print('   class   conf xmin   ymin   xmax   ymax')

            #print(y_pred_decoded)

            #print(time.time() - start_time)


            # Draw the predicted boxes in blue
            xmin = int(inv_yhat[0] / 2)
            ymin = int(inv_yhat[1] / 2)
            xmax = int(inv_yhat[2] / 2)
            ymax = int(inv_yhat[3] / 2)

            print((xmin, ymin))
            print((xmax, ymax))
            # 1920x1080
            #cv2.rectangle(img, (0,  0 ), (1910, 1070), (255, 0, 0), 2)
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax ), (255, 0, 0), 1)

            # Display the resulting frame
            cv2.imshow('frame',img)

            # waitKey: 0, wait indefinitely
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    end_time_video = time.time()
    print("No of frames: {}".format(i))
    print("Total Time: {}".format(total_time))
    print("fps: {}".format(i / (total_time)))

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [98]:
play()

[[0.8  0.67 0.84 0.87]]
[[0.8  0.67 0.84 0.86]
 [0.8  0.67 0.84 0.86]
 [0.8  0.67 0.84 0.87]]
[1513.48  638.23 1604.57  933.35]
(756, 319)
(802, 466)
[[0.8  0.67 0.84 0.87]]
[[0.8  0.67 0.84 0.86]
 [0.8  0.67 0.84 0.87]
 [0.8  0.67 0.84 0.87]]
[1510.91  639.04 1604.74  933.66]
(755, 319)
(802, 466)
[[0.79 0.67 0.84 0.87]]
[[0.8  0.67 0.84 0.87]
 [0.8  0.67 0.84 0.87]
 [0.79 0.67 0.84 0.87]]
[1507.35  640.32 1605.33  933.61]
(753, 320)
(802, 466)
[[0.79 0.67 0.84 0.86]]
[[0.8  0.67 0.84 0.87]
 [0.79 0.67 0.84 0.87]
 [0.79 0.67 0.84 0.86]]
[1503.01  641.94 1606.26  933.23]
(751, 320)
(803, 466)
[[0.79 0.68 0.84 0.86]]
[[0.79 0.67 0.84 0.87]
 [0.79 0.67 0.84 0.86]
 [0.79 0.68 0.84 0.86]]
[1498.14  643.81 1607.42  932.65]
(749, 321)
(803, 466)
[[0.79 0.68 0.84 0.86]]
[[0.79 0.67 0.84 0.86]
 [0.79 0.68 0.84 0.86]
 [0.79 0.68 0.84 0.86]]
[1492.97  645.86 1608.7   931.94]
(746, 322)
(804, 465)
[[0.78 0.68 0.84 0.86]]
[[0.79 0.68 0.84 0.86]
 [0.79 0.68 0.84 0.86]
 [0.78 0.68 0.84 0.86]]
[1487.

[[0.65 0.74 0.69 0.83]]
[[0.66 0.73 0.7  0.83]
 [0.65 0.73 0.7  0.83]
 [0.65 0.74 0.69 0.83]]
[1230.81  701.24 1333.42  890.37]
(615, 350)
(666, 445)
[[0.64 0.74 0.69 0.83]]
[[0.65 0.73 0.7  0.83]
 [0.65 0.74 0.69 0.83]
 [0.64 0.74 0.69 0.83]]
[1220.98  702.45 1323.34  890.37]
(610, 351)
(661, 445)
[[0.64 0.74 0.68 0.83]]
[[0.65 0.74 0.69 0.83]
 [0.64 0.74 0.69 0.83]
 [0.64 0.74 0.68 0.83]]
[1210.89  703.7  1313.02  890.45]
(605, 351)
(656, 445)
[[0.63 0.74 0.68 0.83]]
[[0.64 0.74 0.69 0.83]
 [0.64 0.74 0.68 0.83]
 [0.63 0.74 0.68 0.83]]
[1200.53  704.99 1302.47  890.59]
(600, 352)
(651, 445)
[[0.63 0.74 0.67 0.83]]
[[0.64 0.74 0.68 0.83]
 [0.63 0.74 0.68 0.83]
 [0.63 0.74 0.67 0.83]]
[1189.89  706.33 1291.69  890.81]
(594, 353)
(645, 445)
[[0.62 0.74 0.67 0.83]]
[[0.63 0.74 0.68 0.83]
 [0.63 0.74 0.67 0.83]
 [0.62 0.74 0.67 0.83]]
[1178.98  707.71 1280.67  891.12]
(589, 353)
(640, 445)
[[0.61 0.74 0.66 0.83]]
[[0.63 0.74 0.67 0.83]
 [0.62 0.74 0.67 0.83]
 [0.61 0.74 0.66 0.83]]
[1167.

[[0.28 0.9  0.35 1.02]]
[[0.28 0.9  0.36 1.01]
 [0.28 0.9  0.36 1.02]
 [0.28 0.9  0.35 1.02]]
[ 523.56  857.15  675.54 1104.7 ]
(261, 428)
(337, 552)
[[0.27 0.9  0.35 1.03]]
[[0.28 0.9  0.36 1.02]
 [0.28 0.9  0.35 1.02]
 [0.27 0.9  0.35 1.03]]
[ 514.67  855.72  668.87 1109.81]
(257, 427)
(334, 554)
[[0.27 0.9  0.35 1.03]]
[[0.28 0.9  0.35 1.02]
 [0.27 0.9  0.35 1.03]
 [0.27 0.9  0.35 1.03]]
[ 505.75  853.42  662.23 1114.1 ]
(252, 426)
(331, 557)
[[0.26 0.89 0.34 1.04]]
[[0.27 0.9  0.35 1.03]
 [0.27 0.9  0.35 1.03]
 [0.26 0.89 0.34 1.04]]
[ 496.72  850.35  655.54 1117.51]
(248, 425)
(327, 558)
[[0.26 0.89 0.34 1.04]]
[[0.27 0.9  0.35 1.03]
 [0.26 0.89 0.34 1.04]
 [0.26 0.89 0.34 1.04]]
[ 487.53  846.6   648.74 1120.02]
(243, 423)
(324, 560)
[[0.25 0.88 0.33 1.04]]
[[0.26 0.89 0.34 1.04]
 [0.26 0.89 0.34 1.04]
 [0.25 0.88 0.33 1.04]]
[ 478.14  842.32  641.79 1121.61]
(239, 421)
(320, 560)
[[0.25 0.88 0.33 1.04]]
[[0.26 0.89 0.34 1.04]
 [0.25 0.88 0.33 1.04]
 [0.25 0.88 0.33 1.04]]
[ 468.

[[ -8.59  -8.93  -9.29 -14.18]]
[[ -7.02 -10.63  -6.51 -22.58]
 [-12.29   3.42 -10.73  -4.63]
 [ -8.59  -8.93  -9.29 -14.18]]
[-16323.43  -8508.64 -17826.59 -15301.19]
(-8161, -4254)
(-8913, -7650)
[[-14.96  17.15 -16.67  15.47]]
[[-12.29   3.42 -10.73  -4.63]
 [ -8.59  -8.93  -9.29 -14.18]
 [-14.96  17.15 -16.67  15.47]]
[-28433.12  16342.32 -31995.69  16690.78]
(-14216, 8171)
(-15997, 8345)
[[ 11.18  -8.34  10.52 -10.36]]
[[ -8.59  -8.93  -9.29 -14.18]
 [-14.96  17.15 -16.67  15.47]
 [ 11.18  -8.34  10.52 -10.36]]
[ 21245.73  -7946.55  20183.5  -11173.28]
(10622, -3973)
(10091, -5586)
[[-11.96  -5.06 -13.95  -7.02]]
[[-14.96  17.15 -16.67  15.47]
 [ 11.18  -8.34  10.52 -10.36]
 [-11.96  -5.06 -13.95  -7.02]]
[-22720.01  -4826.14 -26765.79  -7571.16]
(-11360, -2413)
(-13382, -3785)
[[ 13.68 -18.96  14.77 -25.  ]]
[[ 11.18  -8.34  10.52 -10.36]
 [-11.96  -5.06 -13.95  -7.02]
 [ 13.68 -18.96  14.77 -25.  ]]
[ 25982.88 -18067.74  28352.78 -26973.6 ]
(12991, -9033)
(14176, -13486)
[[ -7.3